### 1. Chemical space using PCA and molecular fingerprints


---


In [1]:
from IPython.utils import io
import tqdm.notebook
import os, sys, random
total = 100
with tqdm.notebook.tqdm(total=total) as pbar:
    with io.capture_output() as captured:
      # Instalar rdkit
      !pip -q install rdkit #.pypi==2021.9.4
      pbar.update(20)
      # Instalar Pillow
      !pip -q install Pillow
      pbar.update(40)
      # Instalar molplotly
      !pip install molplotly
      pbar.update(60)
      # Instalar jupyter-dash
      !pip install jupyter-dash
      pbar.update(80)
      # Instalar el diseño de aplicación dash
      !pip install dash-bootstrap-components
      pbar.update(100)

  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
from rdkit.Chem import MACCSkeys
from scipy.spatial.distance import pdist, squareform

In [3]:
# compounds
url_data = "https://drive.google.com/file/d/1DyylcsCIuZ5vzNRr23JS0JY3ELH8exJQ/view?usp=drive_link"
url_data = 'https://drive.google.com/uc?id=' + url_data.split('/')[-2]
data = pd.read_csv(url_data)
data

,ID,isomeric smiles,canonical smiles
0,NPDBEJECOL1,CCCCCC1=CC(=C(C(=C1)O)C2C=C(CCC2C(=C)C)C)O,C=C(C)C1CCC(C)=CC1c1c(O)cc(CCCCC)cc1O
1,NPDBEJECOL2,CCCCCC1=CC(=C2C3C=C(CCC3C(OC2=C1)(C)C)C)O,CCCCCc1cc(O)c2c(c1)OC(C)(C)C1CCC(C)=CC21
2,NPDBEJECOL3,CCCCCC1=CC(=C2C(=C1)OC(C3=C2C=C(C=C3)C)(C)C)O,CCCCCc1cc(O)c2c(c1)OC(C)(C)c1ccc(C)cc1-2
3,NPDBEJECOL4,COC1=C(C=CC(=C1)/C=C/C(=O)CC(=O)/C=C/C2=CC(=C(...,COc1cc(C=CC(=O)CC(=O)C=Cc2ccc(O)c(OC)c2)ccc1O
4,NPDBEJECOL5,COC1=C(C=CC(=C1)/C=C/C(=O)CC(=O)/C=C/C2=CC=C(C...,COc1cc(C=CC(=O)CC(=O)C=Cc2ccc(O)cc2)ccc1O
...,...,...,...
228,NPDBEJECOL232,CN1C=NC2=C1C(=O)N(C(=O)N2C)C,Cn1c(=O)c2c(ncn2C)n(C)c1=O
229,NPDBEJECOL233,CN1C=NC2=C1C(=O)NC(=O)N2C,Cn1cnc2c1c(=O)[nH]c(=O)n2C
230,NPDBEJECOL234,C1=CC(=CC=C1C(=O)NC(CCC(=O)O)C(=O)O)NCC2=CN=C3...,N=c1[nH]c(=O)c2nc(CNc3ccc(C(=O)NC(CCC(=O)O)C(=...
231,NPDBEJECOL235,C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O[C...,O=c1c(O[C@@H]2O[C@H]([C@H](O)CO)[C@H](O)[C@H]2...


## Morgan2 for compounds

---



In [4]:
fps = pd.DataFrame([[int(y) for y in AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x), 2, nBits=1024).ToBitString()] for x in data["canonical smiles"]])
fps

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


In [5]:
# Molecular similarity
SimMat_fps = 1 - pdist(fps[[x for x in range(1024)]], metric="jaccard")#Matriz de similitud_FDA
SimMat_fps
SimMat_fps = squareform(SimMat_fps)
SimMat_fps = 1-SimMat_fps

# Print matrix similarity
SimMat_fps = pd.DataFrame(SimMat_fps)
SimMat_fps

,0,1,2,3,4,5,6,7,8,9,...,223,224,225,226,227,228,229,230,231,232
0,1.000000,0.428571,0.677419,0.838710,0.848485,0.894737,0.857143,0.816667,0.894737,0.872727,...,0.958333,0.900000,0.933333,0.875000,0.936508,0.951613,0.953125,0.923077,0.896552,0.905882
1,0.428571,1.000000,0.580645,0.852941,0.861111,0.887097,0.888889,0.758065,0.904762,0.866667,...,0.943396,0.948276,0.961538,0.909091,0.926471,0.940299,0.957143,0.927835,0.879121,0.887640
2,0.677419,0.580645,1.000000,0.857143,0.865672,0.894737,0.950820,0.907692,0.950000,0.931034,...,0.958333,0.962264,0.956522,0.897959,0.953125,0.951613,0.936508,0.946237,0.870588,0.879518
3,0.838710,0.852941,0.857143,1.000000,0.114286,0.485714,0.897959,0.910714,0.940000,0.960000,...,0.974359,0.902439,0.848485,0.842105,0.924528,0.921569,0.924528,0.900000,0.837838,0.830986
4,0.848485,0.861111,0.865672,0.114286,1.000000,0.371429,0.905660,0.916667,0.944444,0.962963,...,0.976744,0.911111,0.864865,0.857143,0.929825,0.927273,0.929825,0.904762,0.831169,0.824324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.951613,0.940299,0.951613,0.921569,0.927273,0.930233,0.906977,0.920000,0.878049,0.953488,...,0.968750,0.972973,0.931034,0.942857,0.529412,1.000000,0.484848,0.905405,0.932432,0.929577
229,0.953125,0.957143,0.936508,0.924528,0.929825,0.933333,0.911111,0.923077,0.909091,0.955556,...,0.970588,0.974359,0.935484,0.945946,0.700000,0.484848,1.000000,0.863014,0.934211,0.931507
230,0.923077,0.927835,0.946237,0.900000,0.904762,0.887324,0.947368,0.964286,0.946667,0.987013,...,0.984615,0.985714,0.915254,0.923077,0.863014,0.905405,0.863014,1.000000,0.923077,0.920792
231,0.896552,0.879121,0.870588,0.837838,0.831169,0.867647,0.931507,0.937500,0.915493,0.958904,...,0.967742,0.970149,0.966667,0.920635,0.948052,0.932432,0.934211,0.923077,1.000000,0.274194


In [6]:
#SimMat_fps=fps

In [7]:
# Data Normalized
from sklearn.preprocessing import StandardScaler
data_std = SimMat_fps.copy()
data_std = StandardScaler().fit_transform(data_std)

In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(data_std)
pca_results

array([[ 4.00610427e+00,  6.70840734e-01],
       [ 5.59170773e+00, -6.60661201e-01],
       [ 6.34959088e-01, -4.32236581e+00],
       [-3.96459090e+00, -9.02813032e+00],
       [-3.64630339e+00, -8.90209002e+00],
       [-3.09898003e+00, -7.47926849e+00],
       [ 6.11707083e+00,  1.03643576e+00],
       [ 1.48295211e+01,  2.82288730e+00],
       [ 7.44561211e+00,  2.95709478e-01],
       [ 1.29547447e+01,  4.59981358e+00],
       [ 1.40750717e+01,  1.59403103e+00],
       [ 1.31069369e+01,  1.66640691e+00],
       [ 1.35592313e+01,  1.01852350e+00],
       [ 4.93797093e+00, -5.67721927e+00],
       [ 1.11051596e+01,  1.00926020e+00],
       [ 8.92964030e+00,  3.04579361e+00],
       [ 7.46858473e+00,  7.70434733e-01],
       [-4.34880222e+00,  4.85596503e+00],
       [-4.71749549e+00,  9.12337049e+00],
       [-3.47763322e+00,  5.78766195e+00],
       [ 6.10050975e+00,  5.44081323e-02],
       [ 1.26273424e+01,  4.09564005e+00],
       [-6.86877878e+00,  6.40097035e+00],
       [-6.

In [9]:
# Complementary information
label = data[["ID", "canonical smiles"]]
label = label.to_numpy()

In [10]:
# Concat arrays de numpy
arr = np.concatenate((label, pca_results), axis = 1)
# Crreate a new dataframe
pca_dataset = pd.DataFrame(data=arr, columns = ["ID", "SMILES",'component1', 'component2'])
pca_dataset.head(2)

,ID,SMILES,component1,component2
0,NPDBEJECOL1,C=C(C)C1CCC(C)=CC1c1c(O)cc(CCCCC)cc1O,4.006104,0.670841
1,NPDBEJECOL2,CCCCCc1cc(O)c2c(c1)OC(C)(C)C1CCC(C)=CC21,5.591708,-0.660661


# Cumulative variance graph



In [18]:
# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

array([0.3355875 , 0.11127463])

In [19]:
#Graph
import plotly.express as px
import molplotly
fig_pca = px.scatter(pca_dataset,
                            x='component1',
                            y='component2',
                            title='PCA',
                            labels={'component1': f'PC1 {round(exp_var_pca[0] * 100, 2)}%',
                                    'component2': f'PC2 {round(exp_var_pca[1] * 100, 2)}%'},
                            width=700,
                            height=500)

fig_pca.update_traces(marker=dict(color='orange'))

app_marker = molplotly.add_molecules(fig=fig_pca,
                                         df=pca_dataset,
                                         smiles_col='SMILES',
                                         title_col='ID')
#fig_pca.show()
#app_marker.run_server(mode='inline', port=8060, height=1000)
app_marker.run(port=8060)

/usr/local/lib/python3.11/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

In [20]:
import plotly.express as px
import molplotly

fig_pca = px.scatter(pca_dataset,
                     x='component1',
                     y='component2',
                     title='PCA',
                     labels={'component1': f'PC1 {round(exp_var_pca[0] * 100, 2)}%',
                             'component2': f'PC2 {round(exp_var_pca[1] * 100, 2)}%'},
                     width=700,
                     height=500)

fig_pca.update_traces(marker=dict(color='orange'),
                      customdata=pca_dataset[['ID', 'SMILES']],
                      hovertemplate=(
                          'ID: %{customdata[0]}<br>' +
                          'Component 1 (PC1): %{x}<br>' +
                          'Component 2 (PC2): %{y}<br>' +
                          'SMILES: %{customdata[1]}<br>' +
                          '<extra></extra>'
                      ))

app_marker = molplotly.add_molecules(fig=fig_pca,
                                     df=pca_dataset,
                                     smiles_col='SMILES',
                                     title_col='ID')

app_marker.run(port=8060)

fig_pca.write_html("pca_compounds_plot_morgan2.html")

/usr/local/lib/python3.11/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

## MACCS keys for compounds

---



In [21]:
# Recalculate the MACCS keys
fps = pd.DataFrame([[int(y) for y in MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(x)).ToBitString()] for x in data['canonical smiles']])
# Molecular similarity
SimMat_fps = 1 - pdist(fps[[x for x in range(167)]], metric="jaccard")#Matriz de similitud_FDA
SimMat_fps
SimMat_fps = squareform(SimMat_fps)
SimMat_fps = 1-SimMat_fps

# Print matrix similarity
SimMat_fps = pd.DataFrame(SimMat_fps)
SimMat_fps

,0,1,2,3,4,5,6,7,8,9,...,223,224,225,226,227,228,229,230,231,232
0,1.000000,0.309524,0.395349,0.534884,0.558140,0.600000,0.500000,0.658537,0.578947,0.650000,...,0.942857,0.852941,0.861111,0.722222,0.805970,0.787879,0.805970,0.776316,0.636364,0.622642
1,0.309524,1.000000,0.105263,0.625000,0.645833,0.659091,0.536585,0.550000,0.609756,0.575000,...,0.947368,0.894737,0.871795,0.743590,0.761194,0.742424,0.761194,0.753247,0.631579,0.618182
2,0.395349,0.105263,1.000000,0.608696,0.630435,0.642857,0.619048,0.634146,0.589744,0.658537,...,0.972973,0.974359,0.864865,0.729730,0.734375,0.714286,0.734375,0.729730,0.592593,0.576923
3,0.534884,0.625000,0.608696,1.000000,0.034483,0.241379,0.700000,0.913043,0.710526,0.860465,...,0.933333,0.937500,0.800000,0.794118,0.706897,0.706897,0.706897,0.724638,0.478261,0.454545
4,0.558140,0.645833,0.630435,0.034483,1.000000,0.214286,0.725000,0.934783,0.736842,0.883721,...,0.931034,0.935484,0.793103,0.823529,0.724138,0.724138,0.724138,0.720588,0.466667,0.441860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0.787879,0.742424,0.714286,0.706897,0.724138,0.785714,0.830508,0.901639,0.821429,0.918033,...,0.979167,0.980000,0.918367,0.925926,0.042553,1.000000,0.042553,0.561644,0.701493,0.731343
229,0.805970,0.761194,0.734375,0.706897,0.724138,0.785714,0.850000,0.919355,0.842105,0.918033,...,0.979167,0.980000,0.918367,0.925926,0.000000,0.042553,1.000000,0.541667,0.701493,0.731343
230,0.776316,0.753247,0.729730,0.724638,0.720588,0.693548,0.876712,0.933333,0.871429,0.917808,...,0.983607,1.000000,0.900000,0.907692,0.541667,0.561644,0.541667,1.000000,0.571429,0.558824
231,0.636364,0.631579,0.592593,0.478261,0.466667,0.534884,0.836364,0.949153,0.826923,0.929825,...,0.976744,1.000000,0.857143,0.895833,0.701493,0.701493,0.701493,0.571429,1.000000,0.095238


In [22]:
# Data Normalized
from sklearn.preprocessing import StandardScaler
data_std = SimMat_fps.copy()
data_std = StandardScaler().fit_transform(data_std)

In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_results = pca.fit_transform(data_std)
pca_results

array([[  0.8490244 ,   3.50180448],
       [ -1.14210326,   6.18955486],
       [  2.38534311,   4.71215836],
       [ 14.84984525,   3.23885004],
       [ 16.16985271,   3.04213357],
       [ 15.7709344 ,   2.83266586],
       [ -9.26441521,   4.15893151],
       [-11.0794117 ,   7.86814986],
       [ -5.37326656,   1.60135826],
       [ -8.26998725,   8.42141186],
       [-10.93203536,   7.51293252],
       [-11.54657999,   5.51418691],
       [ -2.85130414,  11.03249873],
       [  8.45893706,   6.47767528],
       [ -9.78391938,   7.90453909],
       [ -4.91044   ,   4.04630154],
       [ -8.97597515,   1.28841569],
       [-10.91845208,  -2.58995441],
       [ -8.87758058,  -2.39106302],
       [ -8.91247759,  -3.36683619],
       [  2.97938319,   5.17728781],
       [ -9.35087745,   4.01068419],
       [ -9.22388714,  -0.99447078],
       [ -9.22388714,  -0.99447078],
       [ 11.64768805,   2.56045475],
       [ -9.24978956,   6.49029709],
       [ -3.20844025,   4.39930196],
 

In [24]:
# Complementary information
label = data[["ID", "canonical smiles"]]
label = label.to_numpy()

In [25]:
# Concat arrays de numpy
arr = np.concatenate((label, pca_results), axis = 1)
# Crreate a new dataframe
pca_dataset = pd.DataFrame(data=arr, columns = ["ID", "SMILES",'component1', 'component2'])
pca_dataset.head(2)

,ID,SMILES,component1,component2
0,NPDBEJECOL1,C=C(C)C1CCC(C)=CC1c1c(O)cc(CCCCC)cc1O,0.849024,3.501804
1,NPDBEJECOL2,CCCCCc1cc(O)c2c(c1)OC(C)(C)C1CCC(C)=CC21,-1.142103,6.189555


In [26]:
# Determine explained variance using explained_variance_ration_ attribute
exp_var_pca = pca.explained_variance_ratio_
exp_var_pca

array([0.4068873 , 0.25149934])

In [27]:
#Graph
import plotly.express as px
import molplotly
fig_pca = px.scatter(pca_dataset,
                            x='component1',
                            y='component2',
                            title='PCA',
                            labels={'component1': f'PC1 {round(exp_var_pca[0] * 100, 2)}%',
                                    'component2': f'PC2 {round(exp_var_pca[1] * 100, 2)}%'},
                            width=700,
                            height=500)

fig_pca.update_traces(marker=dict(color='red'))

app_marker = molplotly.add_molecules(fig=fig_pca,
                                         df=pca_dataset,
                                         smiles_col='SMILES',
                                         title_col='ID')
#fig_pca.show()
#app_marker.run_server(mode='inline', port=8060, height=1000)
app_marker.run(port=8060)

/usr/local/lib/python3.11/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

In [29]:
import plotly.express as px
import molplotly

fig_pca = px.scatter(pca_dataset,
                     x='component1',
                     y='component2',
                     title='PCA',
                     labels={'component1': f'PC1 {round(exp_var_pca[0] * 100, 2)}%',
                             'component2': f'PC2 {round(exp_var_pca[1] * 100, 2)}%'},
                     width=700,
                     height=500)

fig_pca.update_traces(marker=dict(color='red'),
                      customdata=pca_dataset[['ID', 'SMILES']],
                      hovertemplate=(
                          'ID: %{customdata[0]}<br>' +
                          'Component 1 (PC1): %{x}<br>' +
                          'Component 2 (PC2): %{y}<br>' +
                          'SMILES: %{customdata[1]}<br>' +
                          '<extra></extra>'
                      ))

app_marker = molplotly.add_molecules(fig=fig_pca,
                                     df=pca_dataset,
                                     smiles_col='SMILES',
                                     title_col='ID')

app_marker.run(port=8060)

fig_pca.write_html("pca_compounds_plot_maccs.html")

/usr/local/lib/python3.11/dist-packages/dash/dash.py:579: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>